In [6]:
#!pip install nltk

In [7]:
import numpy as np
import pandas as pd
from SportsExperiencePlatform.data import connect_db, get_data
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import save_npz, load_npz

In [8]:
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('omw-1.4')

In [9]:
conn = connect_db()

if conn:
    users_df, events_df = get_data(conn)

/home/daniel/.pyenv/versions/3.8.12/envs/SportsExperiencePlatform/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/daniel/.pyenv/versions/3.8.12/envs/SportsExperiencePlatform/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [10]:
users_df.shape, events_df.shape

((9, 13), (155, 17))

In [11]:
users_df.head()

,id,email,encrypted_password,reset_password_token,reset_password_sent_at,remember_created_at,created_at,updated_at,first_name,last_name,description,date_of_birth,gender
0,28,johanna@gmail.com,$2a$12$7BYcXbX4W7rPnDyJbxGJiutNgIlHvAPAHoWwu5z...,None,None,None,2022-05-24 19:14:08.981895,2022-05-24 19:14:08.981895,Johanna,Maier,"Hi, my name is Johanna! I want to go on my nex...",1993-01-25,female
1,29,anjali@gmail.com,$2a$12$//8mGDUQB69ox10kkoXRzeVOc1H8ZWsCTHzVmKE...,None,None,None,2022-05-24 19:14:09.234848,2022-05-24 19:14:09.234848,Anjali,Kumar,"Hi, my name is Anjali! I want to go on my next...",1993-01-25,female
2,30,alexandra@gmail.com,$2a$12$x53r3wSNIfRxv5pzAOcE8eHJJIOkjtiX5QzI9Zs...,None,None,None,2022-05-24 19:14:09.507482,2022-05-24 19:14:09.507482,Alexandra,Stroe,"Hi, my name is Alexandra! I want to go on my n...",1993-01-25,female
3,31,ieva@gmail.com,$2a$12$aYVKb3pm678FGzysHgoSz.X01UkRywVWMcNkW6K...,None,None,None,2022-05-24 19:14:09.771237,2022-05-24 19:14:09.771237,Ieva,Jirgensone,"Hi, my name is Ieva! I want to go on my next s...",1993-01-25,female
4,32,daniel@gmail.com,$2a$12$w4Ej8JIqQ4F4bT8sg8o.cuKmKrI4fhX/8fY1YQL...,None,None,None,2022-05-24 19:14:10.027176,2022-05-24 19:14:10.027176,Daniel,Angerer,"Hi, my name is Daniel! I want to go on my next...",1993-01-25,male


In [12]:
events_df.sample(10)

,id,title,description,price_per_person,capacity,address,latitude,longitude,is_external,url,offer_date,offer_time,created_at,updated_at,user_id,tag_id,external_image_url
35,170,Soccer (Wollishofen or Allmand),We play every Tuesday inside at the moment. Th...,5,NaN,no address provided,47.354008,8.520648,True,https://www.meetup.com/Outdoor-Soccer-spring-a...,2022-06-07,18:30:00,2022-05-24 19:25:59.412588,2022-05-24 19:25:59.412588,None,36,https://secure-content.meetupstatic.com/images...
93,183,Surfcamp Scheveningen: learn to surf and get t...,"here there,\nwanna come surfing with us? I lov...",0,3.0,Strandslag 8,52.080040,4.234879,True,https://www.meetup.com/The-Hague-Rotterdam-Exp...,2022-06-26,11:00:00,2022-05-24 19:26:03.987230,2022-05-24 19:26:03.987230,None,37,https://secure-content.meetupstatic.com/images...
34,125,Wednesday Evening Bouldering @BRIGHT SITE,Hey there 👋\n\nHow about an after-work boulder...,0,NaN,Wilhelm-Kabus-Straße 40,52.481853,13.365493,True,https://www.meetup.com/Berlin-Outdoor-Adventur...,2022-05-25,20:00:00,2022-05-24 19:25:43.908606,2022-05-24 19:25:43.908606,None,33,https://secure-content.meetupstatic.com/images...
91,181,Surfcamp Scheveningen. Get the ride of your li...,"here there,\nwanna come surfing with us? I lov...",0,5.0,Strandslag 8,52.080040,4.234879,True,https://www.meetup.com/NL-Adventurers/events/2...,2022-06-26,11:00:00,2022-05-24 19:26:03.977356,2022-05-24 19:26:03.977356,None,37,https://secure-content.meetupstatic.com/images...
82,173,Bouldering at StudioBloc Darmstadt,"Hey, hey!\nWe are a group of climbers/bouldere...",0,1.0,Gottlieb-Daimler-Straße 6,49.815550,8.610950,True,https://www.meetup.com/Frankfurt-Extremsport-C...,2022-05-26,11:00:00,2022-05-24 19:26:03.934450,2022-05-24 19:26:03.934450,None,37,https://secure-content.meetupstatic.com/images...
21,111,Half-court Pick-up Basketball,Half-court pick-up basketball at Het Marnix. F...,4,NaN,Marnixplein 1,52.378002,4.878494,True,https://www.meetup.com/Pick-up-Basketball-Amst...,2022-05-25,18:00:00,2022-05-24 19:25:38.207598,2022-05-24 19:25:38.207598,None,32,https://secure-content.meetupstatic.com/images...
151,242,Come to play volleyball!,Attention: Please Read the instruction careful...,0,NaN,Vogelzanglaan 6,51.188720,4.397454,True,https://www.meetup.com/Volleyball-every-week-i...,2022-05-28,16:00:00,2022-05-24 19:26:39.693234,2022-05-24 19:26:39.693234,None,40,https://secure-content.meetupstatic.com/images...
7,98,Worlds,"Plato | Know how to listen, and you will profi...",30,8.0,"Suite 373 7268 Mohr Causeway, Lake Marcelo, NM...",-81.624393,73.474483,False,None,2022-06-02,20:08:19,2022-05-24 19:24:35.619620,2022-05-24 19:24:35.633247,None,38,None
52,142,Bouldering at StudioBloc Darmstadt,"Hey, hey!\nWe are a group of climbers/bouldere...",0,1.0,Gottlieb-Daimler-Straße 6,49.815550,8.610950,True,https://www.meetup.com/Frankfurt-Extremsport-C...,2022-05-26,11:00:00,2022-05-24 19:25:48.890993,2022-05-24 19:25:48.890993,None,34,https://secure-content.meetupstatic.com/images...
146,237,☀️🐥 Thu 8:00pm - 2x2 Beach volley training BEG...,Join our one & only international training bea...,14,1.0,Bartákova,50.046940,14.437779,True,https://www.meetup.com/beach-bums-prague/event...,2022-06-09,20:00:00,2022-05-24 19:26:39.668171,2022-05-24 19:26:39.668171,None,40,https://secure-content.meetupstatic.com/images...


In [13]:
def remove_punctuation(text):
    for s in text:
        if s in string.punctuation:
            text = text.replace(s, '')

    return text

In [14]:
def lower_text(text):
    return text.lower()

In [15]:
def remove_numbers(text):
    return ''.join(word for word in text  if not word.isdigit())

In [16]:
def remove_stop_words(text, language='english'):
    stop_words = set(stopwords.words(language))
    word_tokens = word_tokenize(text)
    
    return ' '.join([w for w in word_tokens if not w in stop_words])

In [17]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()

    return ' '.join([lemmatizer.lemmatize(word) for word in text.split(' ')])

In [18]:
events_df['description_punct'] = events_df.description.apply(remove_punctuation)
events_df[['description', 'description_punct']].head()

,description,description_punct
0,Plutarch | Pleasure in the job puts perfection...,Plutarch Pleasure in the job puts perfection ...
1,"Pythagoras | It's not what happens to you, but...",Pythagoras Its not what happens to you but ho...
2,Pythagoras | Rhetoric is the art of ruling the...,Pythagoras Rhetoric is the art of ruling the ...
3,Posidonius | The virtue of justice consists in...,Posidonius The virtue of justice consists in ...
4,Parmenides | Only the educated are free.,Parmenides Only the educated are free


In [19]:
events_df['description_lower'] = events_df.description_punct.apply(lower_text)
events_df[['description', 'description_lower']].head()

,description,description_lower
0,Plutarch | Pleasure in the job puts perfection...,plutarch pleasure in the job puts perfection ...
1,"Pythagoras | It's not what happens to you, but...",pythagoras its not what happens to you but ho...
2,Pythagoras | Rhetoric is the art of ruling the...,pythagoras rhetoric is the art of ruling the ...
3,Posidonius | The virtue of justice consists in...,posidonius the virtue of justice consists in ...
4,Parmenides | Only the educated are free.,parmenides only the educated are free


In [20]:
events_df['description_numbers'] = events_df.description_lower.apply(remove_numbers)
events_df[['description', 'description_numbers']].head()

,description,description_numbers
0,Plutarch | Pleasure in the job puts perfection...,plutarch pleasure in the job puts perfection ...
1,"Pythagoras | It's not what happens to you, but...",pythagoras its not what happens to you but ho...
2,Pythagoras | Rhetoric is the art of ruling the...,pythagoras rhetoric is the art of ruling the ...
3,Posidonius | The virtue of justice consists in...,posidonius the virtue of justice consists in ...
4,Parmenides | Only the educated are free.,parmenides only the educated are free


In [21]:
events_df['description_stopwords'] = events_df.description_numbers.apply(remove_stop_words, language='english')
events_df[['description', 'description_stopwords']].head()

,description,description_stopwords
0,Plutarch | Pleasure in the job puts perfection...,plutarch pleasure job puts perfection work
1,"Pythagoras | It's not what happens to you, but...",pythagoras happens react matters
2,Pythagoras | Rhetoric is the art of ruling the...,pythagoras rhetoric art ruling minds men
3,Posidonius | The virtue of justice consists in...,posidonius virtue justice consists moderation ...
4,Parmenides | Only the educated are free.,parmenides educated free


In [22]:
events_df['description_lemmatize'] = events_df.description_stopwords.apply(lemmatize_text)
events_df[['description', 'description_lemmatize']].head()

,description,description_lemmatize
0,Plutarch | Pleasure in the job puts perfection...,plutarch pleasure job put perfection work
1,"Pythagoras | It's not what happens to you, but...",pythagoras happens react matter
2,Pythagoras | Rhetoric is the art of ruling the...,pythagoras rhetoric art ruling mind men
3,Posidonius | The virtue of justice consists in...,posidonius virtue justice consists moderation ...
4,Parmenides | Only the educated are free.,parmenides educated free


In [23]:
events_df[['description_lemmatize']].to_csv("test.csv")

In [24]:
tfidf = TfidfVectorizer(stop_words="english")

In [25]:
tfidf_matrix = tfidf.fit_transform(events_df["description_lemmatize"])
tfidf_matrix.shape

(155, 1690)

In [26]:
cosine_sim = tfidf_matrix.dot(tfidf_matrix.T)

In [27]:
cosine_sim

<155x155 sparse matrix of type '<class 'numpy.float64'>'
	with 11741 stored elements in Compressed Sparse Row format>

In [28]:
save_npz("model_cosine_sim", cosine_sim)

In [29]:
matrix_reloaded = load_npz("model_cosine_sim.npz")

In [30]:
matrix_reloaded.shape, cosine_sim.shape

((155, 155), (155, 155))

In [31]:
events_df[['description']].duplicated().sum()

79

In [32]:
title_to_index = pd.Series(events_df.index, index = events_df["title"]).drop_duplicates()

In [33]:
def content_recommender(title, cosine_sim = cosine_sim, df = events_df, title_to_index = title_to_index):
    idx = title_to_index[title]
    
    sim_scores = list(enumerate(cosine_sim[idx].todense().tolist()[0]))
    # list of set [(index, sim score), (index, sim score), ...]
    
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    # Sort the list of set by sim score, index with highest simlirarity
    # Will be at the beginning of the list
    
    sim_scores = sim_scores[1:11] # top 10 events (first one is our input events so we ignore it)
    
    sport_indices = [i[0] for i in sim_scores] # we grab the indices of those 10 events
    
    return df["title"].iloc[sport_indices]

In [34]:
events_df.title.unique()

array(['MLG Meadowlands', 'League All Stars', 'Serie A', 'ESL Cologne',
       'GFinity London', 'DFB Pokal', 'Super League', 'Worlds', 'Ligue 1',
       'Copa del Rey', 'Baseball @ Tempelhof',
       'Full court indoor Saturday Basketball in the centre of Rotterdam',
       "Casual Pick-up Women's Basketball",
       'Monday evening Basketball at Bergstr in Berlin Mitte 10115',
       'Basketball Sunday', 'Friday night basketball @ de Pijp',
       'Half-court Pick-up Basketball',
       'Sunday Basketball (INDOOR DOELENPLEIN) 🏀',
       'SPORTEGRATION Basketball', 'Basketball (outdoor)',
       'Monday Night Basketball at Hubelmatt',
       'Bouldering at Boulderwelt Frankfurt',
       'Bouldering at Boulderhaus DA',
       'Bouldering at StudioBloc Darmstadt',
       'Wednesday Evening Bouldering @BRIGHT SITE',
       'Soccer (Wollishofen or Allmand)',
       'Saturday Afternoon Bouldering @BRIGHT SITE',
       'Bouldering at Urban Apes St. Pauli (3G)',
       'Outside Bouldering in

In [36]:
#title = "Surf & Yoga Retreat Scheveningen"
#title = 'Bouldering at StudioBloc Darmstadt'
title = 'Beach Volleyball at Branik - OUTSIDE!'
results = content_recommender(title, cosine_sim = matrix_reloaded, df = events_df, title_to_index = title_to_index)
#content_recommender(title)

In [38]:
results.to_dict()

{41: 'Climbing/Bouldering in Smichov',
 42: 'Climbing/Bouldering in Smichov',
 49: 'Randa Boulder bouldering ',
 50: 'Randa Boulder bouldering ',
 54: 'TRADITIONAL kayaking in Utrecht town and Amelisweerd (more nature part)',
 55: 'Hike (17km) in Mertert/Wasserbillig',
 56: 'TRADITIONAL kayaking in Utrecht town and Amelisweerd (more nature part)',
 59: 'Hike in National Park the Loonse en Drunense Dunes ánd Picnic!',
 60: 'LUXEMBOURG: camping weekend together : Pentecost 2022',
 61: 'Hiking in Germany : Monschau '}